# Multiplier TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=7, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

### fxp & flp2fix : floating number to fixed number conversion

In [6]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [8]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

### LFSR : make pseudorandom number bitstream

In [9]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[5]),int(stream[6]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [10]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [11]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [12]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

### findMaxMin : find abs max value in act tensor

In [13]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

### SNG : stochastic number generator module

In [14]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '1'+(2**(args.bBW))*'0'
    else :
        return sA

In [15]:
lfsr = LFSRlist7()

In [16]:
a = SNG(0.2,lfsr)
print(a)
print(a.count('1'))

n = 0
for i in a :
    if i == '1' :
        n += 1
print(n)
print(len(a))
print(n/(len(a)-1))

000001110000010010010000000000000000000010000010000000000000010000000000000001111001111000111000011000000001100000000000000100001
26
26
129
0.203125


### SNG_P : permutation stochastic number generator module

In [17]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '1'+(2**(args.bBW))*'0'
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [18]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1): # numpy 행렬 중 하나의 요소인 aNumpy
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size()) # Numpy 형태로 출력

### SNGpnumpy : permutation SNG module with numpy

In [19]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [20]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if num[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape) # tensor 형태로 저장하여 출력

### defSign : sign determination in S2N 

In [21]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

### Multiplier

In [22]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [23]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [24]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [25]:
def S2None(sIn): # stochstic number => floating number
    s = defSign(sIn)
#    o = (CountOne(sIn)/(2**args.bBW))*SF*s
    o = sIn*s
    return o

### Accumulator : OUR Scheme

In [26]:
def pos(SN):
    return SN[0] == '0'
def neg(SN):
    return SN[0] == '1'
def counter(x):
    return x.count('1')

In [27]:
def gen_acclist(bBW, IN):
    A_list=[]
    acc_A=0
    acc_Alist=[]
    for i in range(0,2**(bBW-1)):
        A = 0
        for k in range(len(IN)):
            if IN[k][i] == '1':
                A += 1
        A_list.append(A)
        acc_A += A_list[i]
        acc_Alist.append(acc_A)     
    
    return acc_Alist

In [28]:
def gen_So(bBW, diff):
    So_list=[]
    Ao_list=[]
    global So
    for o in range(2**(bBW-1)):
        if o == 0:
            Ao = 0 # first A_o is fixed
            if diff[o] > 0:
                So = 1
            else:
                So = 0
            Ao_list.append(Ao)
            So_list.append(So)
        else:
            Ao += So_list[o-1]
            Ao_list.append(Ao)
            if diff[o] > Ao_list[o]:
                So = 1
            elif diff[o] <= Ao_list[o]:
                So = 0
            So_list.append(So)
    
    return So_list

In [29]:
def block(IN):
    BLOCK1=[]
    BLOCK2=[]
    BLOCK3=[]
    BLOCK4=[]
    BLOCK5=[]
    BLOCK6=[]
    BLOCK7=[]
    BLOCK8=[]
    for i in range(len(IN)):
        block1 = IN[i][1:17]
        block2 = IN[i][17:33]
        block3 = IN[i][33:49]
        block4 = IN[i][49:65]
        block5 = IN[i][65:81]
        block6 = IN[i][81:97]
        block7 = IN[i][97:113]
        block8 = IN[i][113:]
        BLOCK1.append(block1)
        BLOCK2.append(block2)
        BLOCK3.append(block3)
        BLOCK4.append(block4)
        BLOCK5.append(block5)
        BLOCK6.append(block6)
        BLOCK7.append(block7)
        BLOCK8.append(block8)
    return BLOCK1, BLOCK2, BLOCK3, BLOCK4, BLOCK5, BLOCK6, BLOCK7, BLOCK8

In [30]:
def OUR(iA): # bBW = 8
    global diff1, diff2, diff3, diff4, diff5, diff6, diff7, diff8, sign
  
    # sorting random input bit-stream(positive/negative)
    pos_IN = list(filter(pos, iA))
    neg_IN = list(filter(neg, iA))

    # Block devision
    pBLOCK1, pBLOCK2, pBLOCK3, pBLOCK4, pBLOCK5, pBLOCK6, pBLOCK7, pBLOCK8 = block(pos_IN)
    nBLOCK1, nBLOCK2, nBLOCK3, nBLOCK4, nBLOCK5, nBLOCK6, nBLOCK7, nBLOCK8 = block(neg_IN)

    # generate list of number of accumulated 1s
    Ap1_list = gen_acclist(5, pBLOCK1)
    An1_list = gen_acclist(5, nBLOCK1)
    Ap2_list = gen_acclist(5, pBLOCK2)
    An2_list = gen_acclist(5, nBLOCK2)
    Ap3_list = gen_acclist(5, pBLOCK3)
    An3_list = gen_acclist(5, nBLOCK3)
    Ap4_list = gen_acclist(5, pBLOCK4)
    An4_list = gen_acclist(5, nBLOCK4)
    Ap5_list = gen_acclist(5, pBLOCK5)
    An5_list = gen_acclist(5, nBLOCK5)
    Ap6_list = gen_acclist(5, pBLOCK6)
    An6_list = gen_acclist(5, nBLOCK6)
    Ap7_list = gen_acclist(5, pBLOCK7)
    An7_list = gen_acclist(5, nBLOCK7)
    Ap8_list = gen_acclist(5, pBLOCK8)
    An8_list = gen_acclist(5, nBLOCK8)

    # determine sign of output
    if Ap1_list[-1]+Ap2_list[-1]+Ap3_list[-1]+Ap4_list[-1] > An1_list[-1]+An2_list[-1]+An3_list[-1]+An4_list[-1]:
        diff1 = [x-y for x,y in zip(Ap1_list, An1_list)]
        diff2 = [x-y for x,y in zip(Ap2_list, An2_list)]
        diff3 = [x-y for x,y in zip(Ap3_list, An3_list)]
        diff4 = [x-y for x,y in zip(Ap4_list, An4_list)]
        diff5 = [x-y for x,y in zip(Ap5_list, An5_list)]
        diff6 = [x-y for x,y in zip(Ap6_list, An6_list)]
        diff7 = [x-y for x,y in zip(Ap7_list, An7_list)]
        diff8 = [x-y for x,y in zip(Ap8_list, An8_list)]
        sign  = ['0']        
    elif Ap1_list[-1]+Ap2_list[-1]+Ap3_list[-1]+Ap4_list[-1] < An1_list[-1]+An2_list[-1]+An3_list[-1]+An4_list[-1]:
        diff1 = [x-y for x,y in zip(An1_list, Ap1_list)]
        diff2 = [x-y for x,y in zip(An2_list, Ap2_list)]
        diff3 = [x-y for x,y in zip(An3_list, Ap3_list)]
        diff4 = [x-y for x,y in zip(An4_list, Ap4_list)]
        diff5 = [x-y for x,y in zip(An5_list, Ap5_list)]
        diff6 = [x-y for x,y in zip(An6_list, Ap6_list)]
        diff7 = [x-y for x,y in zip(An7_list, Ap7_list)]
        diff8 = [x-y for x,y in zip(An8_list, Ap8_list)]
        sign  = ['1']
    
    So1 = gen_So(5, diff1)
    So2 = gen_So(5, diff2)
    So3 = gen_So(5, diff3)
    So4 = gen_So(5, diff4)
    So5 = gen_So(5, diff5)
    So6 = gen_So(5, diff6)
    So7 = gen_So(5, diff7)
    So8 = gen_So(5, diff8)
    So_list = So1 + So2 + So3 + So4 + So5 + So6 + So7 + So8

    result = list(map(str, So_list))
    tempout = ''.join(result)

    p = abs((Ap1_list[-1]+Ap2_list[-1]+Ap3_list[-1]+Ap4_list[-1]+Ap5_list[-1]+Ap6_list[-1]+Ap7_list[-1]+Ap8_list[-1])-(An1_list[-1]+An2_list[-1]+An3_list[-1]+An4_list[-1]+An5_list[-1]+An6_list[-1]+An7_list[-1]+An8_list[-1]))
    q = counter(tempout)

    p_list=[]
    q_list=[counter(tempout)]
    out=[]
    for k in range(2**7):
        p_list.append(p)
        if q < p:
            q += 1
            out.append('1')
        elif q > p:
            q -= 1
            out.append('0')
        else:
            q = p
            out.append(tempout[k])
        q_list.append(q)
    sout = sign + out
    output = ''.join(sout)
    
    return output

In [31]:
def macNumpy(aIn,bIn,aSF,wSF): # a: input, b: weight
    mList = []
    for i in range(aIn.shape[0]): # 1 
        for j in range(bIn.T.shape[1]): # 16
            for k in range(aIn.shape[1]): # 784
                mList.append(S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str))))
    oList = OUR(mList)
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

In [32]:
def FindError(a,b):
    error = 0
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        error += e
    return error/(len(a.view(-1))+len(b.view(-1)))

# TEST : LFSR 4 bit

In [54]:
act = torch.rand((3,15))-torch.randint(5,(3,15))+torch.randint(5,(3,15))
wei = torch.rand((15,5))-torch.randint(5,(15,5))+torch.randint(5,(15,5))
print(act)
print(wei)

tensor([[ 2.6063, -1.0187,  2.0703, -0.3212, -3.3674, -3.1076, -0.5250, -1.3472,
          4.2343, -0.9597,  0.2342, -0.7310,  0.2126, -1.1464,  1.5389],
        [-0.6839,  2.6538,  1.3520, -0.6460,  3.1900,  1.3727,  1.4249,  1.5117,
          0.5072, -3.6595,  2.6295, -3.2059,  0.2244,  2.5245,  0.0532],
        [ 0.3222,  3.1325, -0.1541, -1.3988,  4.8692,  2.1328, -3.7983, -0.5614,
         -1.7757,  2.9871,  0.3570,  4.9635,  2.0120,  1.6224,  0.5795]])
tensor([[ 0.6500,  1.4406,  0.1038,  1.5654,  1.2554],
        [ 0.3557,  3.4946,  2.2698,  1.8435,  4.2460],
        [ 0.0560, -2.1485,  1.3039,  3.9326, -1.2403],
        [-1.6981, -3.3077,  0.7979, -0.6199, -0.4362],
        [-1.5890, -0.9928,  2.9471, -0.3918,  4.1944],
        [-0.7799, -0.0989, -3.5250,  2.8534,  0.7272],
        [ 1.6771,  3.1918, -0.7986,  1.9996,  1.2360],
        [ 1.3870, -0.2422,  2.3600,  0.3544,  0.2628],
        [ 3.4321,  2.7226,  4.9671, -0.5197, -2.3248],
        [ 0.8825, -0.7088,  2.1082, -2.272

In [55]:
act_f = flp2fixTensor(act,args.full_bits,args.frac_bits)
wei_f = flp2fixTensor(wei,args.full_bits,args.frac_bits)

### number to stochastic conversion

In [56]:
aSF = findMaxMin(act_f)
wSF = findMaxMin(wei_f)

In [57]:
def N2S(act,wei,aSF,wSF):
    
    lfsrlist = LFSRlist7()
    
    a = SNGnumpy(act/aSF,lfsrlist)
    w = SNGpnumpy(wei/wSF,lfsrlist)
    
    return a,w

In [58]:
def N2Sone(data,SF):
    lfsrlist = LFSRlist7()
    
    o = SNGnumpy(data/SF,lfsrlist)
    
    return o

### stochastic to number conversion

In [59]:
def S2N(a,w,aSF,wSF):
    
    sa = defSign(a)
    sw = defSign(w)

    ca = (CountOne(a)/(2**args.bBW)) * aSF *sa
    cw = (CountOne(w)/(2**args.bBW)) * wSF *sw

    return ca,cw

In [60]:
a,w = N2S(act,wei,aSF,wSF)
print(a, a.shape, len(a[0][0]))
print(w, w.shape)

[['011100111011011001001010010000100111001011010001000110011010101000000011111111111110111101011100001101110100110001010110000010111'
  '100001000000000000010000100111001000000000000110011000100000000011111100000000000000000000000000110000000001000110000000011100000'
  '011000011011011001000010000000000011001011010000000010011010101000000001111101111100111101011100000101110100110000010110000000111'
  '100000000000000000000000000001000000000000000000000000000000000000111100000000000000000000000000010000000000000000000000001100000'
  '100111100100100111111111111111111100111110101111111101110101011111111110000010000011100010100011111010001011101111111101111111100'
  '100111100100100111111101111111111100111100101111111101110101010111111110000010000011100010100011111010001011101111101001111111000'
  '100000000000000000000000000011000000000000000010001000000000000001111100000000000000000000000000010000000000000010000000001100000'
  '10001100000000001001000010011101100001000000011001100010000

In [97]:
def multest(aIn,wIn,aSF,wSF):
    mlist=[]
    out = 0
    result=[]
    for a in range(aIn.shape[0]): # 3
        for b in range(wIn.shape[1]): # 5
            acc = []
            for c in range(aIn.shape[1]): #15
#                print(aIn[a][c])
#                print(wIn[c][b])
                acc.append(mul(aIn[a][c].astype(str),wIn[c][b].astype(str)))
#                print(mul(aIn[a][c].astype(str),wIn[c][b].astype(str)))
#                print('----------------------------------------------------------')
            mlist.append(acc)
#    return np.array(mlist).reshape(aIn.shape)
    for i in range(aIn.shape[1]): 
        out = OUR(mlist[i])
        print(f'out = {out}')
        result.append(out)
    print(f'mlist = {mlist}', len(mlist), len(mlist[0])) 
    print(f'result = {result}')
#    return torch.tensor(result).view(aIn.shape[0],wIn.shape[1])
    return np.array(result).reshape(aIn.shape[0],wIn.shape[1])

In [98]:
c = multest(a,w,aSF,wSF) 
print(c)

out = 011111111111111111111111111111111100111111111100001111111111111110011111100011111111111111111111100000001110011111111111100000101
out = 011111111111111111111111111101111100000111101000001101100010001000010111100011111111111111111111100110000111111110111000100000001
out = 100000000000000000011100000010000000000000000111110000000000000000111000000000000001100000000000011111101001111110010000011111110
out = 100000000000000000000000000010010000000100000011010000100000000010111001100000000000000000000000011110000000000000100000011111110
out = 111111111111111111111111111111111111111111111111111111111111111111111111111111100001100000000000111111111111111111111111111111111
out = 100000000000000000000000000000000011111110001010000111011100010000000001111110000000000000000000011111111000100000100110010100110
out = 011111101101011111001000000001001000000000000000001000000000000001111100000000001111111111111111100110000000111010011000000000001
out = 011111111111111111111111111111111000000011

In [ ]:
#cc = flp2fixTensor(S2None(c),args.full_bits,args.frac_bits)

In [ ]:
print(aSF,wSF)

In [ ]:
for i in range(100):
    for j in range(150):
        print(f'act value is            : {act_f[i][j]:^25}(bin : {flp2fix(act[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {a[i][j]}')
        print(f'wei value is            : {wei_f[i][j]:^25}(bin : {flp2fix(wei[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {w[i][j]}')
        print(f'stochastic mul value is : {cc[i][j]:^25}(bin : {flp2fix(cc[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {c[i][j]}')
        print(f'fixed mul value is      : {act_f[i][j]*wei_f[i][j]:^25}(bin : {flp2fix(act[i][j]*wei[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {N2Sone(act[i][j]*wei[i][j],aSF*wSF)}')
        print(f'------------------------------------------------------------------------------------------')

In [ ]:
def MSE(a,b):
    s = 0
    n = 0

    for i in range(100):
        for j in range(150):
            square = (((a[i][j])-b[i][j])**2).item()
            s += square
            n += 1
    error = s/n
    return error

## ERROR : 8bit LFSR

In [ ]:
MSE(act_f*wei_f,cc)